In [1]:
import boto3
iam = boto3.client('iam')
response = iam.list_roles()
sagemaker_roles = [role for role in response['Roles'] if 'SageMaker' in role['RoleName']]
sagemaker_roles[0]['RoleName']

'AmazonSageMaker-ExecutionRole-20231030T210397'

In [2]:
import sagemaker
import boto3
sess = sagemaker.Session()
sagemaker_session_bucket=None
if sagemaker_session_bucket is None and sess is not None:
    sagemaker_session_bucket = sess.default_bucket()
try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName='AmazonSageMaker-ExecutionRole-20231030T210397')['Role']['Arn']
sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)
print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {sess.boto_region_name}")

sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/ravi.tej/Library/Application Support/sagemaker/config.yaml


Couldn't call 'get_role' to get Role ARN from role name ravi_tej to get Role path.


sagemaker role arn: arn:aws:iam::005418323977:role/service-role/AmazonSageMaker-ExecutionRole-20231030T210397
sagemaker bucket: sagemaker-ap-south-1-005418323977
sagemaker session region: ap-south-1


In [3]:
import pandas as pd
import numpy as np
import json

import re
from transformers import AutoTokenizer
from random import randint
import sys
sys.path.append("../utils")
from pack_dataset import pack_dataset
from datasets import Dataset

import requests

In [4]:
tokenizer_model_id = 'teknium/OpenHermes-2.5-Mistral-7B'

In [5]:
# model_id = "TheBloke/OpenHermes-2.5-Mistral-7B-AWQ"
tokenizer = AutoTokenizer.from_pretrained(tokenizer_model_id, trust_remote_code=True)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [6]:
cleaned_responses = pd.read_csv('cleaned_gpt_responses-6-feb.csv')

In [8]:
new_system_prompt = '''You are the chief editor for an indian finance and business portal. You answer valid inquiries using relevant articles you've found for the query. Valid queries pertain to finance, business, or entities like India, RBI, Reliance, Fed, etc. All other queries are invalid. You encounter 3 scenarios with specific response guidelines:
Case 1: The query is valid and answerable with available articles.
Response instructions:
* Provide answers with a headline, then 3 to 6 points, each starting with an emoji and a bolded label, followed by the point.
* Format: <Headline \n\n emoji1 label1: point1 \n\n emoji2 label2: point2 \n\n…>. Begin with a headline, then list points. Separate points with \n\n.
* Each point must be brief, answer a unique aspect of the query, and highly relevant.
* Begin with the most important point. Points collectively should fully address the query.
* Numbers and dates in points must match those in the articles.
* The Emoji and the label should be highly relevant to the point and the query. Use only professional emojis. No underscores or programming symbols in labels.
* Headlines should be 5 to 15 words, tailored to the query.
Case 2: The query is valid, but no exact answer is found in the articles.
Response instructions: Apologize and state that you are not able to find the exact answer
Case 3: The query is invalid.
Response instructions: Politely decline, noting the focus on finance and business.
General Guidelines:
*Only use information from the articles.
*If the article is dated, mention the period; if very recent or from today, no need to specify.
*Be concise and direct in responses, following the instructions based on the query's case and the articles. Relevant articles to answer the query are in JSON format: {article_0: {‘title’: ‘’, ‘published_date’: ‘’, ‘content’: ‘’} || ...}'''

In [9]:
len(tokenizer.encode(new_system_prompt))

458

In [10]:
tokenizer.encode('\n')

[1, 28705, 13]

In [11]:
tokenizer.encode('<br>')

[1, 523, 1473, 28767]

### Formatting the responses to \n from <br>

In [12]:
cleaned_responses['new_response'] = cleaned_responses['modified_response'].apply(lambda x: x.replace('<br>','\n'))

In [16]:
cleaned_responses['last_line'] = cleaned_responses['new_response'].apply(lambda x: x.split('\n')[-1])

In [21]:
'a b c'.split(' ')[:-1]

['a', 'b']

### Removing disclaimers and lines without emoji and label

We remove disclaimer lines at the end

In [79]:
def extract_content_disclaimer(response):
    response = response.strip('\n\n')
    first_part = '\n\n'.join(response.split('\n\n')[:-1])
    last_line = response.split('\n\n')[-1].lower()
    if 'please' in last_line or 'investment' in last_line or 'research' in last_line:
        return first_part, last_line, 'disclaimer'
    if '**' in last_line:
        return response, None, 'point'
    if 'sorry' in last_line or 'apologize':
        if '**' in response:
            return first_part, last_line, 'partly_point'
        else:
            return response, None, 'sorry'
    else:
        return response, 'what?', 'unk'

In [80]:
cleaned_responses['truncated_response'], cleaned_responses['disclaimer'], cleaned_responses['last_line_type'] = zip(*cleaned_responses['new_response'].apply(lambda x: extract_content_disclaimer(x)))

In [81]:
cleaned_responses.groupby('last_line_type')['last_line_type'].count()

last_line_type
disclaimer      1893
partly_point      64
point            348
sorry            175
Name: last_line_type, dtype: int64

In [77]:
cleaned_responses[cleaned_responses.last_line_type == 'disclaimer'].sample(20).last_line.values

array(["While the articles provide a comprehensive outlook on the vaccination campaigns and health initiatives announced in India's Union Budget 2024, users should do additional thorough research and consult a healthcare expert for personal medical advice.",
       'Note: As investment decisions should be made carefully, please ensure to do additional thorough research and consult an expert.',
       'Users interested in ACC shares or the cement industry should conduct further research and consult an expert for investment advice.',
       'Make sure to do additional thorough research and consult an expert before making investment decisions.',
       '⚠️ **Investment Caution**: Consider additional research and consult an expert before investing.',
       "Please note, it's critical to do further research and consult a financial expert before making investment decisions.",
       'Please do additional thorough research and consult an expert for investment-related advice.',
       'I\'m s

In [82]:
cleaned_responses[cleaned_responses.last_line_type == 'point'].sample(20).truncated_response.values

array(["**Decline in Automotive Sales Amidst Market Fluctuations**\n\n📉 **Sales Drop**: Automotive Axle net sales down 17.45% YoY in Dec 2023.\n\n🚗 **M&M Sales Dip**: M&M auto sales see a 4% decline in January '24 from last year.\n\n📊 **Industry High**: Indian auto registers highest monthly PV wholesales in Jan '24.\n\n📈 **SUV Demand**: SUV sales now over 50% of total auto sales, a significant increase.\n\n🚀 **Hyundai Peak**: Hyundai domestic sales up by 14% YoY, marking record monthly sales.\n\n💹 **Supply Recovery**: Supply constraints easing, semiconductor chip issues resolved for India.",
       '**Understanding the Business of Artisan Bread: The Baker’s Dozen Expansion**\n\n**💰 Funding Milestone**: Raised $4M for expansion.\n\n**🏙️ Urban Spread**: Plans to reach beyond metro cities.\n\n**📈 Business Growth**: Aiming for an INR 500 Cr company in five years.\n\n**🍞 Product Enhancement**: Enhancing mass-premium segment.\n\n**📊 Previous Growth**: Secured $3M in 2022, doubled ARR since.\

In [86]:
cleaned_responses[cleaned_responses.last_line_type == 'sorry'].sample(20).truncated_response.values

array(['I\'m sorry, but it seems like the user query about "paragliding safety" doesn\'t align with the financial and business scope of our platform. Our focus is on providing insights and answers related to finance, business, and their relevant entities. If you have any questions on those topics, feel free to ask!',
       "I'm sorry, but your query does not fall within the scope of finance and business that our platform covers. If you have any questions related to these areas, feel free to ask, and I would be happy to help.",
       "I'm sorry, but your query about digital detox strategies is not related to finance or business topics that we cover on our platform. We focus on providing expert content in fields like finance, business and economics. If you have any questions within those realms, feel free to ask!",
       "I'm sorry, but your query about environmental conservation talks doesn't pertain to the financial and business content covered on our platform. We focus exclusively 

In [89]:
cleaned_responses[cleaned_responses.last_line_type == 'partly_point'].sample(20).last_line.values

array(["I'm not able to find the exact answer for this, but found the related information.",
       "I'm sorry, but it seems your question is related to public health, which falls outside the scope of financial and business topics that we cover. We focus on finance and business-related questions and would be unable to provide an analysis or summary of public health information. If you have any finance or business-related questions, feel free to ask!",
       "I'm sorry but we don't have the latest unemployment data. However, the related information indicates significant changes in the job market and economic growth which might impact employment figures.",
       "I'm not able to find the exact answer for this, but found the related information from the IEEE Computer Society's 2024 technology predictions that may interest you.",
       "Unfortunately, the articles available don't cover details regarding Kidambi Srikanth's matches. My resources are limited to financial and business conte

In [95]:
cleaned_responses.sample(20).disclaimer.values

array(['please ensure to do additional thorough research and to consult an expert before making any investment decisions.',
       None,
       'for investment advice, remember to do additional thorough research and to consult an expert.',
       'investment advice should be sought from experts, as it requires additional thorough research.',
       'please do further research or consult with an insurance expert for a detailed comparison tailored to individual needs.',
       "please note that while this information provides a snapshot of the current investment atmosphere, it's important to conduct thorough research and consult a financial expert before making any investment decisions.",
       "it's important to do additional thorough research and consult an expert when considering investment decisions.",
       None,
       'always remember to perform thorough research and consider consulting a financial expert before making any investment decisions.',
       "investment advice can be

In [96]:
cleaned_responses.truncated_response.sample(20).values

array(['**Optimism Abounds for MSMEs and Startup Growth in 2024**\n\n💼 **MSME Confidence**: 90% confident in India\'s economic resilience, expecting profit boost in 2024.\n\n💵 **Profit Forecast**: Bengaluru, Pune MSMEs predict over 30% profits; Mumbai less optimistic with predictions under 30%.\n\n📈 **Expansion Plans**: 60% of small businesses plan new stores and products/services in 2024.\n\n🚀 **Hiring Trends**: 44% of MSMEs to increase hiring; emphasis on diversity and opportunities for people with disabilities.\n\n🌏 **Sustainable Practices**: 81% of MSMEs aim for eco-friendly initiatives in 2024, continuing the plastic-free trend.\n\n📊 **Credit Growth**: Approximately 60% foresee a need for credit, with the highest demand in Bangalore, Pune, and Delhi.\n\nWhile the information provided does not directly outline specific "small business growth strategies today," it indicates the overarching optimism and strategic planning of Indian MSMEs that could be essential practices for growth. 

### Generating new data

In [102]:
cleaned_responses.to_csv('cleaned_gpt_responses-9-feb-no-disclaimer.csv',index=False)

In [98]:
cleaned_responses.head(30)

,0,index,query,query_date,article_ids,gpt_prompt,response,system_prompt,user_prompt,modified_response,has_our_site,is_sorry,new_response,last_line,truncated_response,disclaimer,last_line_type
0,0,4,social media influencers,16/02/24,"['65a7c2a01c9bc64bb8ae4a12', '65a7b6f11c9bc64b...",You're the lead editor for a financial and bus...,"I'm sorry, but the queries you've provided rel...",You're the lead editor for a financial and bus...,|| user_query: social media influencers || tod...,"I'm sorry, but the queries you've provided rel...",True,True,"I'm sorry, but the queries you've provided rel...","I'm sorry, but the queries you've provided rel...","I'm sorry, but the queries you've provided rel...",None,sorry
1,0,5,kotak mahindra bank's banking and financial se...,23/12/23,"['65a662593c864051fa47ec9e', '65a9e5d56391ac19...",You're the lead editor for a financial and bus...,**Kotak Mahindra Bank Navigates Funding and Gr...,You're the lead editor for a financial and bus...,|| user_query: kotak mahindra bank's banking a...,**Kotak Mahindra Bank Navigates Funding and Gr...,False,False,**Kotak Mahindra Bank Navigates Funding and Gr...,"**Investment Note**: For investment decisions,...",**Kotak Mahindra Bank Navigates Funding and Gr...,"**investment note**: for investment decisions,...",disclaimer
2,0,1,security token offerings news,05/02/24,"['65b320421b79671c6b70cca2', '65a91cba6391ac19...",You're the lead editor for a financial and bus...,**Exploring Recent Security Token Offerings an...,You're the lead editor for a financial and bus...,|| user_query: security token offerings news |...,**Exploring Recent Security Token Offerings an...,False,False,**Exploring Recent Security Token Offerings an...,Please remember to do additional thorough rese...,**Exploring Recent Security Token Offerings an...,please remember to do additional thorough rese...,disclaimer
3,0,7,rock climbing basics,28/12/23,"['65bb79c3f3aa2a77a58bfe62', '65bb79cef3aa2a77...",You're the lead editor for a financial and bus...,**Understanding the Impact of Budget 2024 on I...,You're the lead editor for a financial and bus...,|| user_query: rock climbing basics || today d...,**Understanding the Impact of Budget 2024 on I...,False,False,**Understanding the Impact of Budget 2024 on I...,*Articles from early 2024 highlight expectatio...,**Understanding the Impact of Budget 2024 on I...,*articles from early 2024 highlight expectatio...,disclaimer
4,0,2,gold investment,01/01/24,"['65bb096df3aa2a77a58bfb1a', '65a896681c9bc64b...",You're the lead editor for a financial and bus...,**Gold Investment Outlook for 2024 Shows Promi...,You're the lead editor for a financial and bus...,|| user_query: gold investment || today date: ...,**Gold Investment Outlook for 2024 Shows Promi...,False,False,**Gold Investment Outlook for 2024 Shows Promi...,It is important to do thorough research and co...,**Gold Investment Outlook for 2024 Shows Promi...,it is important to do thorough research and co...,disclaimer
5,0,0,regulatory frameworks,27/01/24,"['65b9b7d4f3aa2a77a58bf57f', '65b4aa001b79671c...",You're the lead editor for a financial and bus...,**Decoding Regulatory Frameworks: Data Protect...,You're the lead editor for a financial and bus...,|| user_query: regulatory frameworks || today ...,**Decoding Regulatory Frameworks: Data Protect...,False,False,**Decoding Regulatory Frameworks: Data Protect...,Please note that investing in digital lending ...,**Decoding Regulatory Frameworks: Data Protect...,please note that investing in digital lending ...,disclaimer
6,0,9,bcci,18/01/24,"['65a894451c9bc64bb8ae4ac6', '659e08404b25ce48...",You're the lead editor for a financial and bus...,**BCCI Secures New Partnerships Amid Continued...,You're the lead editor for a financial and bus...,|| user_query: bcci || today date: 18 January ...,**BCCI Secures New Partnerships Amid Continued...,False,False,**BCCI Secures New Partnerships Amid Continued...,Please note that financial decisions should be...,**BCCI S

In [97]:
def format_as_prompt_response(response_row):
    messages = [{"role": "system", "content": new_system_prompt},
                {"role": "user", "content": response_row['user_prompt']}]
    context_prompt = tokenizer.decode(tokenizer.apply_chat_template(messages, add_generation_prompt=True))
    prompt = context_prompt + response_row['truncated_response']
    prompt = re.sub(r'\n+','\n',prompt)
    return prompt

In [101]:
len(tokenizer.encode(format_as_prompt_response(cleaned_responses.iloc[1])))

2719

In [103]:
from datasets import Dataset
sys.path.append("../utils")
from pack_dataset import pack_dataset
finetune_id = 'PinkSparrowOnTV'

In [108]:
train_df = cleaned_responses[['user_prompt','truncated_response']]
dataset = Dataset.from_pandas(train_df)

In [107]:
import os
os.environ['TOKENIZERS_PARALLELISM'] = 'False'

In [109]:
# template dataset to add prompt to each sample
def template_dataset(sample):
    sample["text"] = f"{format_as_prompt_response(sample)}{tokenizer.eos_token}"
    return sample

dataset = dataset.map(template_dataset)
# tokenize dataset
dataset = dataset.map(
    lambda sample: tokenizer(sample["text"]), batched=True, remove_columns=list(dataset.features)
)
# chunk dataset
lm_dataset = pack_dataset(dataset, chunk_length=4096) # We use 4096 as the maximum length for packing
print(f"Total number of samples: {len(lm_dataset)}")

Map:   0%|          | 0/2480 [00:00<?, ? examples/s]

Map:   0%|          | 0/2480 [00:00<?, ? examples/s]

Chunking dataset into chunks of 4096 tokens.


Map:   0%|          | 0/2480 [00:00<?, ? examples/s]

Total number of samples: 1818
Total number of samples: 1818


In [110]:
import sagemaker
import boto3
sess = sagemaker.Session()
sagemaker_session_bucket=None
if sagemaker_session_bucket is None and sess is not None:
    sagemaker_session_bucket = sess.default_bucket()
try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName='AmazonSageMaker-ExecutionRole-20231030T210397')['Role']['Arn']
sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)
print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {sess.boto_region_name}")

Couldn't call 'get_role' to get Role ARN from role name ravi_tej to get Role path.


sagemaker role arn: arn:aws:iam::005418323977:role/service-role/AmazonSageMaker-ExecutionRole-20231030T210397
sagemaker bucket: sagemaker-ap-south-1-005418323977
sagemaker session region: ap-south-1


In [111]:
from datetime import datetime

In [112]:
len(train_df)

2480

In [113]:
finetune_dataset_config = {'finetune_id': 'search_openhermes_fp16_' + finetune_id,
                          'date': datetime.strftime(datetime.today(),'%Y-%m-%d'),
                          'num_datapoints': len(train_df),
                            'data_source': 'gpt4'}

In [114]:
# save train_dataset to s3
training_input_path = f's3://{sess.default_bucket()}/fine_tuning_datasets/{finetune_dataset_config["date"]}-{finetune_dataset_config["finetune_id"]}'
lm_dataset.save_to_disk(training_input_path)

print("uploaded data to:")
print(f"training dataset to: {training_input_path}")

Saving the dataset (0/1 shards):   0%|          | 0/1818 [00:00<?, ? examples/s]

uploaded data to:
training dataset to: s3://sagemaker-ap-south-1-005418323977/fine_tuning_datasets/2024-02-09-search_openhermes_fp16_PinkSparrowOnTV
